# LDA and Document Similarity 
We are using Gensim's LDA model and diff method.

## Loading all the libraries in

In [163]:
# Importing all lib 
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
from gensim import corpora, models
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import common_corpus, common_dictionary

##SKLearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


## spacy for lemmatization
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load()

## Plotting tools
import pyLDAvis
import pyLDAvis.gensim  ## don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

## Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Preprocessing
## First part of the preprocessing will be importing the Data set from GitHub, and loading it into a dataframe.

In [164]:
# Import Dataset
#df = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2016.json')
df1 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2015.json')
df2 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2014.json')
df3 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2013.json')
df4 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2012.json')
df5 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2011.json')
df6 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2010.json')
df7 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2009.json')
df8 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2008.json')
df9 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2006.json')

print(df1.abstracts.unique())
df.head()

['A formal qualitative risk management approach for IT security Information technology (IT) security, which is concerned about protecting the confidentiality, integrity and availability of information technology assets, inherently possesses a significant amount of risk, some known and some unknown. IT security risk management has gained considerable attention over the past decade due to the collapsing of some large organisations in the world. Previous investigative research in the field of IT security have indicated that despite the efforts that organisations employ to reduce IT security risks, the trend of IT security attacks are still increasing. One of the contributing factors to poor management of IT security risk is attributed to the fact that IT security risk management is often left to the technical security technologist who do not necessarily employ formal risk management tools and reasoning. For this reason, organisations find themselves in a position where they do not have th

,abstracts,title
0,Team Formation in Digital Forensics A major pr...,Team Formation in Digital Forensics


### Removing unwanted characters
\begin{itemize}
\item Convert to list
\item Remove Email prefixes
\item Remove New lines
\item Replace Ligature characters
\item Remove single quotes
\end{itemize}



In [165]:
#data1 = df.abstracts.values.tolist()
data2 = df1.abstracts.values.tolist()
data3 = df2.abstracts.values.tolist()
data4 = df3.abstracts.values.tolist()
data5 = df4.abstracts.values.tolist()
data6 = df5.abstracts.values.tolist()
data7 = df6.abstracts.values.tolist()
data8 = df7.abstracts.values.tolist()
data9 = df8.abstracts.values.tolist()
data10 = df9.abstracts.values.tolist()

data = list(set(data2 + data3 + data4+ data5+ data6+ data7+ data8+ data9+ data10))

## Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

## Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

## Remove fi ligature characters
data = [re.sub('ﬁ', 'fi', sent) for sent in data]

## Remove fi ligature characters
data = [re.sub('ﬂ', 'fl', sent) for sent in data]

## Remove single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[1:])

['Secure Publish-Subscribe Mediated Virtual Organizations Digital technologies '
 'such as publish-subscribe systems present dynamic services support for '
 'inter-organizational activities. In order for these systems to achieve usage '
 'acceptance, various security requirements have to be met by the enabling '
 'technologies. In this article, we focus on confidentiality, privacy and '
 'integrity requirements for Publishers and Subscribers in a Publish-Subscribe '
 'mediated electronic market. We consider a virtual organization architecture, '
 'in which subscribers dynamically join and leave various organizations. We '
 'review techniques previously suggested in literature for providing '
 'confidentiality, privacy and integrity requirements and then present a new '
 'solution which is based on cryptographic hashes and public-key cryptography',
 'Data Aggregation Using Homomorphic Encryption in Wireless Sensor Networks '
 'Secure information aggregation using homomorphic encryption 

 'challenge arising due to the novelty and openness of the solutions. We '
 'analyze the security risks of an envisioned e-health application for elderly '
 'persons’ day-to-day support and chronic disease self-care, from the '
 'perspectives of the service provider and end-user. In addition, we propose '
 'initial heuristics for security objective decomposition aimed at security '
 'metrics definition. Systematically defined and managed security metrics '
 'enable higher effectiveness of security controls, enabling informed '
 'risk-driven security decision-making',
 'Access Control for Local Personal Smart Spaces As computer systems grow more '
 'compact, powerful and cheap to produce they become more pervasive in '
 'society. Smart devices enable users to compute and share resources on the '
 'go. Services such as Wi-Fi Direct allow for the creation of device-to-device '
 'networks of a peer-to-peer nature deemed smart spaces. Smart spaces are '
 'capable of providing an accesspoint

 'and deployed on an Android smartphone. In addition, this new Android '
 'application is also evaluated against mobile security applications. From the '
 'results additional security steps are identified that users of smartphones '
 'can follow to prevent or detect possible mobile malware infections. The '
 'ultimate goal of this research is to eventually automate the identified '
 'steps in the form of an application rather than depending on the user to '
 'execute the steps.',
 'AUTHORISED TRANSLATIONS OF ELECTRONIC DOCUMENTS A concept is proposed to '
 'extend authorised translations of documents to electronically signed, '
 'digital documents. Central element of the solution is an electronic seal, '
 'embodied as an XML data structure, which attests to the correctness of the '
 'translation and the authorisation of the translator. The seal contains a '
 'digital signature binding together original and translated document, thus '
 'enabling forensic inspection and therefore legal s

 'lives. One can receive email on a PC, Laptop or mobile phone. SMTP was '
 'designed to be an easy and cost-effective implementation. This fact, '
 'however, makes SMTP a target to be abused. Unsolicited electronic '
 'communication, also known as spam, is just one such example of abuse of '
 'email. Tracing the origin of spam by using the information contained in SMTP '
 'headers is not possible because SMTP is a clear text protocol and can easily '
 'be intercepted and modified. Digital forensic specialists are plagued with '
 'sifting through large data sets to find incident information. During the '
 'process of introducing digital forensic readiness the amount of information '
 'that is gathered is inadvertently increased, to ensure that the information '
 'is valid and usable. Drawing from the experience of digital forensic experts '
 'to find specific data subsets that prove or disprove that an incident '
 'occurred can be used to automate the analysis process. Data analysis to

 'content analysis. These parameters include URL length, number of delimiters '
 'as well as the number of traversals through the directory structure and are '
 'used throughout much of the research in the paradigm of lightweight '
 'classification. Methods which include external sources of information are '
 'often called fully featured classifications and have been shown to be only '
 'slightly more effective than a purely lexical analysis when considering both '
 'false-positives and false-negatives. This distinction allows these '
 'algorithms to be run client side without the introduction of additional '
 'latency, but still providing a high level of accuracy through the use of '
 'modern techniques in training classifiers. Analysis of this type will also '
 'be useful in an incident response analysis where large numbers of URLs need '
 'to be filtered for potentially malicious URLs as an initial step in '
 'information gathering as well as end user implementations such as browser

 'algorithms available. This paper presents four basic properties for '
 'similarity preserving hash functions that are partly related to the '
 'properties of cryptographic hash functions. Compression and ease of '
 'computation are borrowed from traditional hash functions and define the hash '
 'value length and the performance. As every byte is expected to influence the '
 'hash value, we introduce coverage. Similarity score describes the need for a '
 'comparison function for hash values. We shortly discuss these properties '
 'with respect to three existing approaches and finally have a detailed view '
 'on the promising approach sdhash. However, we uncovered some bugs and other '
 'peculiarities of the implementation of sdhash. Finally we conclude that '
 'sdhash has the potential to be a robust similarity preserving digest '
 'algorithm, but there are some points that need to be improved',
 'TOWARDS A FRAMEWORK FOR A NETWORK WARFARE CAPABILITY Information warfare has '
 'surface

 'phone is motivated by the fact that Android mobile phones have the greatest '
 'share of the mobile market index. In the last three years, for example, the '
 'market share index for mobile phones put Android mobile devices at 75% of '
 'the entire smartphone market. Through observing the findings of the test '
 'using an Android mobile phone, this paper demonstrates that conducting '
 'mobile forensics using the HDFI process model produces satisfactory results.',
 'An Exploration of Geolocation and Traffic Visualisation Using Network Flows '
 'A network flow is a data record that represents characteristics associated '
 'with a unidirectional stream of packetstransmitted between two hosts using '
 'an IP layer protocol. Asa network flow only represents statistics relating '
 'to the data transferred in the stream, the effectiveness of utilizing '
 'network flows for traffic visualization to aid in cyber defense is not '
 'immediately apparent and needs further exploration. The goal 

 'transmission of encrypted data), many of them are much more complex (e.g., '
 'ebanking and e-voting applications, or anonymous authentication and '
 'credential systems). For a software engineer without appropriate '
 'cryptographic skills the implementation of such protocols is often '
 'difficult, time consuming and error-prone. For this reason, a number of '
 'compilers supporting programmers have been published in recent years. '
 'However, they are either designed for very specific cryptographic primitives '
 '(e.g., zero-knowledge proofs of knowledge), or they only offer a very low '
 'level of abstraction and thus again demand substantial mathematical and '
 'cryptographic skills from the programmer. Finally, some of the existing '
 'compilers do not produce executable code, but only metacode which has to be '
 'instantiated with mathematical libraries, encryption routines, etc. before '
 'it can actually be used. In this paper we present a cryptographically aware '
 'compile

 'advancing. Developments in digital forensics still leave a lot to be desired '
 'in terms of standards and appropriate digital forensic tools that are '
 'applicable in the Cloud. To achieve that, standards as well as standard '
 'tools are required for successful evidence collection, preservation, '
 'analysis and conviction in case of a criminal case. This paper contributes '
 'towards addressing issues in digital forensics by presenting an algorithm '
 'that can be used in the evidence identification phase of a digital forensic '
 'process. Data in Cloud environments exist in the Internet or in networked '
 'environments and data is always accessed remotely. There is therefore at '
 'least one connection to a host that exists in a Cloud environment. In a case '
 'of a computer system that hosts a Cloud service, the number of connections '
 'from clients can be very large. In such a scenario it is very hard to '
 'identify an attacker from both active and recently disconnected conn

# Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.



In [166]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), min_len=2 ,deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

pprint(data_words[:1])

[['exploring',
  'the',
  'human',
  'dimension',
  'of',
  'tetra',
  'in',
  'order',
  'to',
  'secure',
  'communication',
  'in',
  'the',
  'year',
  'bc',
  'julius',
  'caesar',
  'developed',
  'the',
  'caesar',
  'cipher',
  'to',
  'ensure',
  'his',
  'generals',
  'on',
  'the',
  'battle',
  'field',
  'received',
  'critical',
  'information',
  'in',
  'the',
  'most',
  'secure',
  'manner',
  'possible',
  'today',
  'the',
  'protection',
  'of',
  'critical',
  'information',
  'and',
  'communication',
  'is',
  'just',
  'as',
  'vital',
  'police',
  'officers',
  'fire',
  'marshals',
  'and',
  'emergency',
  'medical',
  'units',
  'require',
  'critical',
  'information',
  'to',
  'make',
  'decisions',
  'that',
  'could',
  'mean',
  'the',
  'difference',
  'between',
  'life',
  'and',
  'death',
  'just',
  'as',
  'in',
  'the',
  'caesar',
  'era',
  'the',
  'information',
  'is',
  'intended',
  'for',
  'particular',
  'recipient',
  'and',
  'can

### Building bigrams and trigrams

In [167]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [252]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['unfortunately','huge', 'used','may','using','jam','found','plays','however','information','security','good','aodv','would','derive','hence','focus','research','present','motive','study','result','within','propose','different','therefore','approach','form','also','new','context','df','pron','opportunity','institution','esc','process','author', 'work', 'base', 'easy','material','area','produce','period', 'publish','end','perform','less','become','seem','possible','vital','tool','be','make', 'move', 'space', 'uses','use','from', 'often','definition','ensure','much','useful','a','s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain','t','all','allow','allows','almost','alone','along','already','also,although,always,am,among,amongst,an,and,another,any,anybody,anyhow,anyone,anything,anyway,anyways,anywhere,apart,appear,appreciate,appropriate,are,aren,t,around,as,aside,ask,asking,associated,at,available,away,awfully,be,became,because,become,becomes,becoming,been,before,beforehand,behind,being,believe,below,beside,besides,best,better,between,beyond,both,brief,but,by,c,mon,c,s,came,can,can,t,cannot,cant,cause,causes,certain,certainly,changes,clearly,co,com,come,comes,concerning,consequently,consider,considering,contain,containing,contains,corresponding,could,couldn,t,course,currently,definitely,described,despite,did,didn,t,different,do,does,doesn,t,doing,don,t,done,down,downwards,during,each,edu,eg,eight,either,else,elsewhere,enough,entirely,especially,et,etc,even,ever,every,everybody,everyone,everything,everywhere,ex,exactly,example,except,far,few,fifth,first,five,followed,following,follows,for,former,formerly,forth,four,from,further,furthermore,get,gets,getting,given,gives,go,goes,going,gone,got,gotten,greetings,had,hadn,t,happens,hardly,has,hasn,t,have,haven,t,having,he,he,s,hello,help,hence,her,here,here,s,hereafter,hereby,herein,hereupon,hers,herself,hi,him,himself,his,hither,hopefully,how,howbeit,however,i,d,i,ll,i,m,i,ve,ie,if,ignored,immediate,in,inasmuch,inc,indeed,indicate,indicated,indicates,inner,insofar,instead,into,inward,is,isn,t,it,it,d,it,ll,it,s,its,itself,just,keep,keeps,kept,know,knows,known,last,lately,later,latter,latterly,least,less,lest,let,let,s,like,liked,likely,little,look,looking,looks,ltd,mainly,many,may,maybe,me,mean,meanwhile,merely,might,more,moreover,most,mostly,much,must,my,myself,name,namely,nd,near,nearly,necessary,need,needs,neither,never,nevertheless,new,next,nine,no,nobody,non,none,noone,nor,normally,not,nothing,novel,now,nowhere,obviously,of,off,often,oh,ok,okay,old,on,once,one,ones,only,onto,or,other,others,otherwise,ought,our,ours,ourselves,out,outside,over,overall,own,particular,particularly,per,perhaps,placed,please,plus,possible,presumably,probably,provides,que,quite,qv,rather,rd,re,really,reasonably,regarding,regardless,regards,relatively,respectively,right,said,same,saw,say,saying,says,second,secondly,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,shall,she,should,shouldn,t,since,six,so,some,somebody,somehow,someone,something,sometime,sometimes,somewhat,somewhere,soon,sorry,specified,specify,specifying,still,sub,such,sup,sure,t,s,take,taken,tell,tends,th,than,thank,thanks,thanx,that,that,s,thats,the,their,theirs,them,themselves,then,thence,there,there,s,thereafter,thereby,therefore,therein,theres,thereupon,these,they,they,d,they,ll,they,re,they,ve,think,third,this,thorough,thoroughly,those,though,three,through,throughout,thru,thus,to,together,too,took,toward,towards,tried,tries,truly,try,trying,twice,two,un,under,unfortunately,unless,unlikely,until,unto,up,upon,us,use,used,useful,uses,using,usually,value,various,very,via,viz,vs,want,wants,was,wasn,t,way,we,we,d,we,ll,we,re,we,ve,welcome,well,went,were,weren,t,what,what,s,whatever,when,whence,whenever,where,where,s,whereafter,whereas,whereby,wherein,whereupon,wherever,whether,which,while,whither,who,who,s,whoever,whole,whom,whose,why,will,willing,wish,with,within,without,won,t,wonder,would,would,wouldn,t,yes,yet,you,you,d,you,ll,you,re,you,ve,your,something,yours,yourself,yourselves,zero,the,of,and,to,is,that,was,for,it,in,om,it,i,it,that,is,in,and,of,to,a,the','subject', 're', 'edu','value', 'is', 'prahp', 'dfr', 'ecfr', 'help', 'current', 'number', 'show', 'find', 'ddo', 'day','site','paper'])
pprint(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [253]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



pprint(data_lemmatized[:1])

['explore human dimension tetra order secure communication year bc julius '
 'caesar develop caesar cipher general battle field receive critical secure '
 'manner today protection critical communication police officer fire marshal '
 'emergency medical unit require critical decision could mean difference life '
 'death caesar era intend particular recipient lead devastating consequence '
 'intercept receive malform state communication utmost importance threat '
 'social engineering commonly deploy computer network social engineering need '
 'limited computer network terrestrial trunked radio tetra standard intend '
 'secure professional digital mobile radio communication design government '
 'agency emergency service transportation service various user need secure '
 'communication tetra see technically secure standard making authentication '
 'key air interface encryption technique even tetra technically secure tetra '
 'user safe social engineering investigate potential social engine

In [254]:
# Building bigrams and trigrams
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops, min_count=2, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
pprint(trigram_mod[bigram_mod[data_words_nostops[0]]])




/Users/juandrevanheerden/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['exploring',
 'human_dimension',
 'tetra',
 'order',
 'secure',
 'communication',
 'year',
 'bc',
 'julius',
 'caesar',
 'developed',
 'caesar',
 'cipher',
 'generals',
 'battle',
 'field',
 'received',
 'critical',
 'secure',
 'manner',
 'today',
 'protection',
 'critical',
 'communication',
 'police',
 'officers',
 'fire',
 'marshals',
 'emergency',
 'medical',
 'units',
 'require',
 'critical',
 'decisions',
 'could',
 'mean',
 'difference',
 'life',
 'death',
 'caesar',
 'era',
 'intended',
 'particular',
 'recipient',
 'lead',
 'devastating',
 'consequences',
 'intercepted',
 'received',
 'malformed',
 'state',
 'communications',
 'utmost_importance',
 'one',
 'threats',
 'social_engineering',
 'commonly',
 'deployed',
 'computer_networks',
 'social_engineering',
 'need',
 'limited',
 'computer_networks',
 'terrestrial',
 'trunked',
 'radio',
 'tetra',
 'standard',
 'intended',
 'secure',
 'professional',
 'digital',
 'mobile',
 'radio',
 'communication',
 'designed',
 'governmen

In [255]:
# Creating dictionary, corpus, and TD-F
## Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

## Create Corpus
texts = data_words_nostops

## Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

## View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 4), (11, 1), (12, 2), (13, 1), (14, 2), (15, 3), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 5), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 6), (74, 1), (75, 2), (76, 5), (77, 2), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 7), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 3), (91, 1), (92, 2), (93, 1), (94, 1)]]


In [256]:
## Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('agencies', 1),
  ('air', 1),
  ('amongst', 1),
  ('authentication', 1),
  ('aware', 1),
  ('battle', 1),
  ('bc', 1),
  ('caesar', 3),
  ('cipher', 1),
  ('commonly', 1),
  ('communication', 4),
  ('communications', 1),
  ('computer', 2),
  ('consequences', 1),
  ('could', 2),
  ('critical', 3),
  ('death', 1),
  ('decisions', 1),
  ('deployed', 1),
  ('designed', 1),
  ('devastating', 1),
  ('developed', 1),
  ('difference', 1),
  ('digital', 1),
  ('dimension', 1),
  ('emergency', 2),
  ('encryption', 1),
  ('engineering', 5),
  ('era', 1),
  ('escalation', 1),
  ('even', 1),
  ('explored', 1),
  ('exploring', 1),
  ('field', 1),
  ('fire', 1),
  ('generals', 1),
  ('government', 1),
  ('human', 1),
  ('importance', 1),
  ('intended', 2),
  ('intercepted', 1),
  ('interface', 1),
  ('investigate', 1),
  ('julius', 1),
  ('keys', 1),
  ('lead', 1),
  ('life', 1),
  ('limited', 1),
  ('made', 1),
  ('making', 1),
  ('malformed', 1),
  ('manner', 1),
  ('marshals', 1),
  ('mean', 1)

In [257]:
# Building LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=np.random.RandomState(0),
                                           iterations = 300,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [258]:
#import pickle
# Save
#lda_model.save("training.pkl")  # Stores *.pkl file
# Load
#lda = pickle.load("training.pkl")

In [259]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.038*"biometric" + 0.036*"ict" + 0.028*"characteristics" + '
  '0.018*"technical" + 0.015*"biometrics" + 0.013*"safety" + 0.011*"among" + '
  '0.010*"important" + 0.010*"inherent" + 0.009*"partners"'),
 (1,
  '0.045*"response" + 0.020*"presentation" + 0.015*"standards" + '
  '0.014*"products" + 0.011*"patch" + 0.010*"captured" + 0.009*"provide" + '
  '0.009*"path" + 0.008*"drm" + 0.006*"category"'),
 (2,
  '0.026*"network" + 0.026*"architecture" + 0.017*"protocols" + '
  '0.015*"intrusion" + 0.015*"networks" + 0.014*"time" + 0.014*"algorithm" + '
  '0.014*"traffic" + 0.013*"attacks" + 0.012*"detection"'),
 (3,
  '0.018*"event" + 0.017*"data" + 0.015*"strategies" + 0.015*"email" + '
  '0.015*"database" + 0.014*"robust" + 0.011*"protection" + '
  '0.010*"operational" + 0.010*"trace" + 0.010*"cloud"'),
 (4,
  '0.040*"online" + 0.025*"awareness" + 0.016*"survey" + 0.013*"warfare" + '
  '0.012*"increasingly" + 0.011*"experience" + 0.011*"mobile" + '
  '0.011*"increasing" + 0.009*"t

In [260]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
19     12.736021        1       1 -0.183152  0.040124
12      9.606359        1       2 -0.144772 -0.094963
5       9.284832        1       3 -0.123269 -0.046181
15      7.552125        1       4 -0.100977 -0.150148
10      7.511082        1       5 -0.089884  0.138132
11      5.877306        1       6 -0.046692  0.216664
18      5.096632        1       7 -0.058987  0.005366
8       4.812690        1       8 -0.051225  0.030053
6       4.518062        1       9 -0.043519  0.041891
14      4.498151        1      10  0.048110  0.078030
2       4.065596        1      11 -0.043261 -0.151697
1       3.762375        1      12  0.209272  0.038786
0       3.416852        1      13  0.040948  0.070442
3       3.404641        1      14  0.024795  0.034624
7       2.982050        1      15  0.170538 -0.085507
13      2.981698        1      16  0.048088 -0.098232
4       2.955716        1      17  0.075419  0.019179
16      1.963973        1      18  0.061236 -0.080700
17      1.556397        1      19  0.109511  0.013402
9       1.417441        1      20  0.097821 -0.019266, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
23    Default  203.000000          digital  203.000000  30.0000  30.0000
423   Default  150.000000         forensic  150.000000  29.0000  29.0000
421   Default  103.000000         evidence  103.000000  28.0000  28.0000
123   Default   84.000000          privacy   84.000000  27.0000  27.0000
758   Default  112.000000            cloud  112.000000  26.0000  26.0000
144   Default  136.000000             data  136.000000  25.0000  25.0000
3465  Default   45.000000         response   45.000000  24.0000  24.0000
259   Default   59.000000            south   59.000000  23.0000  23.0000
898   Default   91.000000             risk   91.000000  22.0000  22.0000
55    Default   51.000000           mobile   51.000000  21.0000  21.0000
224   Default   72.000000    investigation   72.000000  20.0000  20.0000
81    Default   69.000000       techniques   69.000000  19.0000  19.0000
239   Default   39.000000           online   39.000000  18.0000  18.0000
2264  Default   41.000000          malware   41.000000  17.0000  17.0000
3748  Default   37.000000         commerce   37.000000  16.0000  16.0000
158   Default   70.000000          network   70.000000  15.0000  15.0000
76    Default   71.000000           social   71.000000  14.0000  14.0000
100   Default  106.000000            based  106.000000  13.0000  13.0000
187   Default   46.000000        awareness   46.000000  12.0000  12.0000
2501  Default   32.000000        biometric   32.000000  11.0000  11.0000
2525  Default   45.000000        assurance   45.000000  10.0000  10.0000
659   Default   65.000000            risks   65.000000   9.0000   9.0000
2407  Default   31.000000              ict   31.000000   8.0000   8.0000
646   Default   89.000000       management   89.000000   7.0000   7.0000
699   Default   52.000000       compliance   52.000000   6.0000   6.0000
183   Default   74.000000         analysis   74.000000   5.0000   5.0000
387   Default  130.000000            model  130.000000   4.0000   4.0000
253   Default   62.000000          service   62.000000   3.0000   3.0000
1104  Default   40.000000  characteristics   40.000000   2.0000   2.0000
504   Default   53.000000             user   53.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
4636  Topic20    1.923396      illustrated    2.701065   3.9168  -5.2072
4639  Topic20    1.923396         volatile    2.701065   3.9168  -5.2072
4634  Topic20    1.923396        encourage    2.701065   3.9168  -5.2072
4637  Topic20    1.923396    pervasiveness    2.701065   3.9168  -5.2072
4638  Topic20    1.923396           stores    2.701065   3.9168  -5.

# Nuwe dokument

In [261]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2016single.json')


print(df.abstracts.unique())
df.head()

['Team Formation in Digital Forensics A major problem in Digital Forensics (DF) is the often huge volumes of data that has to be searched, filtered, and indexed to discover patterns that could lead to forensic evidence. The nature of, and the process by which the data was collected also means that the data contain information about persons that are not involved, or only incidentally involved in the crime under investigation. Privacy is therefore another potential issue that needs to be dealt with in a DF investigation. This paper shows that techniques of the Team Formation (TF) task can be used to address both of these problems. The TF task can be formulated to fit the DF arena: to commit a crime, the culprit(s) may require the assistance of several other individuals, which implies that a team of some sort gets established. During a post-mortem DF analysis, an investigator may only have one, or a few names to start with. One of the key challenges is finding possible co-conspirators. Fr

,abstracts,title
0,Team Formation in Digital Forensics A major pr...,Team Formation in Digital Forensics


In [262]:
data1 = df.abstracts.values.tolist()




## Remove Emails
data1 = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

## Remove new line characters
data1 = [re.sub('\s+', ' ', sent) for sent in data]

## Remove fi ligature characters
data1 = [re.sub('ﬁ', 'fi', sent) for sent in data]

## Remove fi ligature characters
data1 = [re.sub('ﬂ', 'fl', sent) for sent in data]

## Remove single quotes
data1 = [re.sub("\'", "", sent) for sent in data]

pprint(data1[1:])

['Secure Publish-Subscribe Mediated Virtual Organizations Digital technologies '
 'such as publish-subscribe systems present dynamic services support for '
 'inter-organizational activities. In order for these systems to achieve usage '
 'acceptance, various security requirements have to be met by the enabling '
 'technologies. In this article, we focus on confidentiality, privacy and '
 'integrity requirements for Publishers and Subscribers in a Publish-Subscribe '
 'mediated electronic market. We consider a virtual organization architecture, '
 'in which subscribers dynamically join and leave various organizations. We '
 'review techniques previously suggested in literature for providing '
 'confidentiality, privacy and integrity requirements and then present a new '
 'solution which is based on cryptographic hashes and public-key cryptography',
 'Data Aggregation Using Homomorphic Encryption in Wireless Sensor Networks '
 'Secure information aggregation using homomorphic encryption 

 'the cloud, streamlining the responsibilities of the computing or IT '
 'departments of organisations. IdMaaS‘s attractiveness leans on reduced cost '
 'of ownership, least to no capital investment, scalability, self-service, '
 'location independence and rapid deployment, however, its growth has been '
 'impeded by issues most of which are related to security, privacy and trust. '
 'Most organisations view identities as passports to key computing resources '
 '(hardware, software and data) as such they view identity management as a '
 'core IT function which must remain within the perimeter of sphere of '
 'control. This paper primarily aims to discuss IdMaaS and highlight the major '
 'trust issues in current existing cloud computing environments affecting the '
 'growth of IdMaaS by describing IdMaaS and surveying the trust issues that '
 'pose threats to its growth. Highlighting the trust issues hampering the '
 'growth of IdMaaS will lay a foundation for subsequent research effor

 'play an important role in any information security environment and may '
 'influence security behaviour significantly. In this paper the results of a '
 'practical phishing exercise and a trust survey are considered. The research '
 'project is part of a larger project and the phishing exercise is a follow-up '
 'to an earlier first practical phishing test. Results of the phishing test '
 'are compared with the first exercise. In addition, the newly obtained trust '
 'information from the survey is also incorporated into the report in order to '
 'try and explain security behaviour. The research was performed at a large '
 'organisation. Results indicate that although there is a general high level '
 'of trust in the organisation’s ability to provide safe and secure '
 'information systems, a large number of staff was still victim to a simple '
 'phishing exercise. A possible explanation, which opens up further avenues '
 'for research, is offered.',
 'Forensic Entropy Analysis of Mi

 'Internet was used for commercial purposes. These include the Merchandise '
 'Marks Act of 1941 and Copyright Act of 1978. The findings of this study '
 'reflect that information security practitioners were not really familiar '
 'with the avalanche of information security related legislation. Be that as '
 'it may, the contents of the IT policies from some of the organisations that '
 'participated in this study contain the provisions of legislation were '
 'catered for in the policies. This should be attributed to the fact that '
 'although information security practitioners were not consciously trying to '
 'comply with legislation, they relied heavily on the international standards. '
 'Most of these standards are in line with the requirements of the South '
 'African information security related legislation. In other words, corporate '
 'information security policies are within the framework of the Constitution '
 'of the Republic and the applicable legislation by default. They a

 'which can prepare and prevent incidents from occurring within an eSC '
 'environment if implemented strategically. With the current state of '
 'cybercrime, tool developers are challenged with the task of developing '
 'cutting edge digital forensic readiness tools that can keep up with the '
 'current technological advancements, such as (eSCs), in the business world. '
 'Therefore, the problem addressed in this paper is that there are no DFR '
 'tools that are designed to support eSCs specifically. There are some '
 'general-purpose monitoring tools that have forensic readiness functionality, '
 'but currently there are no tools specifically designed to serve the eSC '
 'environment. Therefore, this paper discusses the limitations of current '
 'digital forensic readiness tools for the eSC environment and an '
 'architectural design for next-generation eSC DFR systems is proposed, along '
 'with the system requirements that such systems must satisfy. It is the view '
 'of the author

In [263]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), min_len=2 ,deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data1))

pprint(data_words[:1])

[['exploring',
  'the',
  'human',
  'dimension',
  'of',
  'tetra',
  'in',
  'order',
  'to',
  'secure',
  'communication',
  'in',
  'the',
  'year',
  'bc',
  'julius',
  'caesar',
  'developed',
  'the',
  'caesar',
  'cipher',
  'to',
  'ensure',
  'his',
  'generals',
  'on',
  'the',
  'battle',
  'field',
  'received',
  'critical',
  'information',
  'in',
  'the',
  'most',
  'secure',
  'manner',
  'possible',
  'today',
  'the',
  'protection',
  'of',
  'critical',
  'information',
  'and',
  'communication',
  'is',
  'just',
  'as',
  'vital',
  'police',
  'officers',
  'fire',
  'marshals',
  'and',
  'emergency',
  'medical',
  'units',
  'require',
  'critical',
  'information',
  'to',
  'make',
  'decisions',
  'that',
  'could',
  'mean',
  'the',
  'difference',
  'between',
  'life',
  'and',
  'death',
  'just',
  'as',
  'in',
  'the',
  'caesar',
  'era',
  'the',
  'information',
  'is',
  'intended',
  'for',
  'particular',
  'recipient',
  'and',
  'can

In [264]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [265]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['huge','unfortunately','used','may','using','jam','found','plays','however','information','security','good','aodv','would','derive','hence','focus','research','present','motive','study','result','within','propose','different','therefore','approach','form','also','new','context','df','pron','opportunity','institution','esc','process','author', 'work', 'base', 'easy','material','area','produce','period', 'publish','end','perform','less','become','seem','possible','vital','tool','be','make', 'move', 'space', 'uses','use','from', 'often','definition','ensure','much','useful','a','s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain','t','all','allow','allows','almost','alone','along','already','also,although,always,am,among,amongst,an,and,another,any,anybody,anyhow,anyone,anything,anyway,anyways,anywhere,apart,appear,appreciate,appropriate,are,aren,t,around,as,aside,ask,asking,associated,at,available,away,awfully,be,became,because,become,becomes,becoming,been,before,beforehand,behind,being,believe,below,beside,besides,best,better,between,beyond,both,brief,but,by,c,mon,c,s,came,can,can,t,cannot,cant,cause,causes,certain,certainly,changes,clearly,co,com,come,comes,concerning,consequently,consider,considering,contain,containing,contains,corresponding,could,couldn,t,course,currently,definitely,described,despite,did,didn,t,different,do,does,doesn,t,doing,don,t,done,down,downwards,during,each,edu,eg,eight,either,else,elsewhere,enough,entirely,especially,et,etc,even,ever,every,everybody,everyone,everything,everywhere,ex,exactly,example,except,far,few,fifth,first,five,followed,following,follows,for,former,formerly,forth,four,from,further,furthermore,get,gets,getting,given,gives,go,goes,going,gone,got,gotten,greetings,had,hadn,t,happens,hardly,has,hasn,t,have,haven,t,having,he,he,s,hello,help,hence,her,here,here,s,hereafter,hereby,herein,hereupon,hers,herself,hi,him,himself,his,hither,hopefully,how,howbeit,however,i,d,i,ll,i,m,i,ve,ie,if,ignored,immediate,in,inasmuch,inc,indeed,indicate,indicated,indicates,inner,insofar,instead,into,inward,is,isn,t,it,it,d,it,ll,it,s,its,itself,just,keep,keeps,kept,know,knows,known,last,lately,later,latter,latterly,least,less,lest,let,let,s,like,liked,likely,little,look,looking,looks,ltd,mainly,many,may,maybe,me,mean,meanwhile,merely,might,more,moreover,most,mostly,much,must,my,myself,name,namely,nd,near,nearly,necessary,need,needs,neither,never,nevertheless,new,next,nine,no,nobody,non,none,noone,nor,normally,not,nothing,novel,now,nowhere,obviously,of,off,often,oh,ok,okay,old,on,once,one,ones,only,onto,or,other,others,otherwise,ought,our,ours,ourselves,out,outside,over,overall,own,particular,particularly,per,perhaps,placed,please,plus,possible,presumably,probably,provides,que,quite,qv,rather,rd,re,really,reasonably,regarding,regardless,regards,relatively,respectively,right,said,same,saw,say,saying,says,second,secondly,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,shall,she,should,shouldn,t,since,six,so,some,somebody,somehow,someone,something,sometime,sometimes,somewhat,somewhere,soon,sorry,specified,specify,specifying,still,sub,such,sup,sure,t,s,take,taken,tell,tends,th,than,thank,thanks,thanx,that,that,s,thats,the,their,theirs,them,themselves,then,thence,there,there,s,thereafter,thereby,therefore,therein,theres,thereupon,these,they,they,d,they,ll,they,re,they,ve,think,third,this,thorough,thoroughly,those,though,three,through,throughout,thru,thus,to,together,too,took,toward,towards,tried,tries,truly,try,trying,twice,two,un,under,unfortunately,unless,unlikely,until,unto,up,upon,us,use,used,useful,uses,using,usually,value,various,very,via,viz,vs,want,wants,was,wasn,t,way,we,we,d,we,ll,we,re,we,ve,welcome,well,went,were,weren,t,what,what,s,whatever,when,whence,whenever,where,where,s,whereafter,whereas,whereby,wherein,whereupon,wherever,whether,which,while,whither,who,who,s,whoever,whole,whom,whose,why,will,willing,wish,with,within,without,won,t,wonder,would,would,wouldn,t,yes,yet,you,you,d,you,ll,you,re,you,ve,your,something,yours,yourself,yourselves,zero,the,of,and,to,is,that,was,for,it,in,om,it,i,it,that,is,in,and,of,to,a,the','subject', 're', 'edu','value', 'is', 'prahp', 'dfr', 'ecfr', 'help', 'current', 'number', 'show', 'find', 'ddo', 'day','site','paper'])
pprint(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [266]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



pprint(data_lemmatized[:1])

['explore human dimension tetra order secure communication year bc julius '
 'caesar develop caesar cipher general battle field receive critical secure '
 'manner today protection critical communication police officer fire marshal '
 'emergency medical unit require critical decision could mean difference life '
 'death caesar era intend particular recipient lead devastating consequence '
 'intercept receive malform state communication utmost importance threat '
 'social engineering commonly deploy computer network social engineering need '
 'limited computer network terrestrial trunked radio tetra standard intend '
 'secure professional digital mobile radio communication design government '
 'agency emergency service transportation service various user need secure '
 'communication tetra see technically secure standard making authentication '
 'key air interface encryption technique even tetra technically secure tetra '
 'user safe social engineering investigate potential social engine

In [267]:
# Building bigrams and trigrams
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops, min_count=1, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
pprint(trigram_mod[bigram_mod[data_words_nostops[0]]])




/Users/juandrevanheerden/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['exploring',
 'human_dimension',
 'tetra',
 'order',
 'secure',
 'communication',
 'year',
 'bc',
 'julius',
 'caesar',
 'developed',
 'caesar',
 'cipher',
 'generals',
 'battle',
 'field',
 'received',
 'critical',
 'secure',
 'manner',
 'today',
 'protection',
 'critical',
 'communication',
 'police',
 'officers',
 'fire',
 'marshals',
 'emergency',
 'medical',
 'units',
 'require',
 'critical',
 'decisions',
 'could',
 'mean',
 'difference',
 'life',
 'death',
 'caesar',
 'era',
 'intended',
 'particular',
 'recipient',
 'lead',
 'devastating',
 'consequences',
 'intercepted',
 'received',
 'malformed',
 'state',
 'communications',
 'utmost_importance',
 'one',
 'threats',
 'social_engineering',
 'commonly',
 'deployed',
 'computer_networks',
 'social_engineering',
 'need',
 'limited',
 'computer_networks',
 'terrestrial',
 'trunked',
 'radio',
 'tetra',
 'standard',
 'intended',
 'secure',
 'professional',
 'digital',
 'mobile',
 'radio',
 'communication',
 'designed',
 'governmen

In [268]:
# Creating dictionary, corpus, and TD-F
## Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

## Create Corpus
texts = data_words_nostops

## Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

## View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 4), (11, 1), (12, 2), (13, 1), (14, 2), (15, 3), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 5), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 6), (74, 1), (75, 2), (76, 5), (77, 2), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 7), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 3), (91, 1), (92, 2), (93, 1), (94, 1)]]


In [269]:
## Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('agencies', 1),
  ('air', 1),
  ('amongst', 1),
  ('authentication', 1),
  ('aware', 1),
  ('battle', 1),
  ('bc', 1),
  ('caesar', 3),
  ('cipher', 1),
  ('commonly', 1),
  ('communication', 4),
  ('communications', 1),
  ('computer', 2),
  ('consequences', 1),
  ('could', 2),
  ('critical', 3),
  ('death', 1),
  ('decisions', 1),
  ('deployed', 1),
  ('designed', 1),
  ('devastating', 1),
  ('developed', 1),
  ('difference', 1),
  ('digital', 1),
  ('dimension', 1),
  ('emergency', 2),
  ('encryption', 1),
  ('engineering', 5),
  ('era', 1),
  ('escalation', 1),
  ('even', 1),
  ('explored', 1),
  ('exploring', 1),
  ('field', 1),
  ('fire', 1),
  ('generals', 1),
  ('government', 1),
  ('human', 1),
  ('importance', 1),
  ('intended', 2),
  ('intercepted', 1),
  ('interface', 1),
  ('investigate', 1),
  ('julius', 1),
  ('keys', 1),
  ('lead', 1),
  ('life', 1),
  ('limited', 1),
  ('made', 1),
  ('making', 1),
  ('malformed', 1),
  ('manner', 1),
  ('marshals', 1),
  ('mean', 1)

In [270]:
# Building LDA model
lda1_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1,
                                           random_state=np.random.RandomState(0),
                                           iterations = 300,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [271]:
pprint(lda1_model.print_topics())


[(0,
  '0.008*"digital" + 0.006*"forensic" + 0.005*"data" + 0.005*"model" + '
  '0.004*"based" + 0.004*"cloud" + 0.004*"system" + 0.004*"evidence" + '
  '0.004*"systems" + 0.004*"framework"')]


In [272]:
# Gensim diff

In [273]:
diff_model = lda1_model.diff(lda_model, distance='hellinger', num_words=10, n_ann_terms=30, diagonal=False, annotation=True, normed=True)

In [274]:
pprint(diff_model)

(array([[0.95277111, 1.        , 0.92344847, 0.89836233, 0.89967999,
        0.88479443, 0.90872393, 0.98847832, 0.91867975, 0.8515358 ,
        0.89508453, 0.94420329, 0.92092589, 0.9248134 , 0.94280608,
        0.90210585, 0.92530555, 0.88922683, 0.8840382 , 0.90463743]]),
 array([[list([[], ['biometrics', 'system', 'ict', 'systems', 'inherent', 'data', 'forensic', 'evidence', 'characteristics', 'framework', 'partners', 'technical', 'safety', 'among', 'model', 'cloud', 'biometric', 'important', 'digital', 'based']]),
        list([[], ['systems', 'response', 'standards', 'path', 'data', 'drm', 'forensic', 'evidence', 'framework', 'presentation', 'cloud', 'products', 'captured', 'system', 'patch', 'model', 'provide', 'category', 'digital', 'based']]),
        list([[], ['attacks', 'algorithm', 'systems', 'data', 'traffic', 'detection', 'forensic', 'evidence', 'framework', 'protocols', 'time', 'network', 'model', 'cloud', 'intrusion', 'system', 'networks', 'architecture', 'digital', 'b